In [69]:
from bs4 import BeautifulSoup
import requests
import csv, time, json
from datetime import datetime

import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('bond_funds_2022_08_15.csv')
data.tail()

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
37,KTPLUS,28457268639,1.0,1.0,1.07,False,0.14,0.04,0.19,0.53,0.80,11/08/2565
38,KTSS,28902119433,0.0,0.0,0.70,False,0.05,0.08,0.13,0.35,0.65,11/08/2565
39,KT-ST,8585282719,0.0,0.0,0.43,False,0.23,0.14,0.34,0.98,1.28,11/08/2565
40,KTSTPLUS-A,32243801140,3.0,3.0,2.14,False,0.17,0.00,0.13,0.75,1.09,11/08/2565
41,KTSV,6813639430,0.0,0.0,0.80,False,0.06,0.12,0.24,0.37,0.62,11/08/2565


In [3]:
df = data.copy()

In [98]:
bond_funds = [
    ('SCBTMFPLUS-P', 'SCBAM-MMF-SCBTMFPLUS%20P-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน%20พลัส%20ชนิดหน่วยลงทุน%20P'),
    ('SCBTMFPLUS-I', 'SCBAM-MMF-SCBTMFPLUS%20I-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน%20พลัส%20ชนิดหน่วยลงทุน%20I'),
    ('SCBTMFPLUS-E', 'SCBAM-MMF-SCBTMFPLUS%20E-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน%20พลัส%20ชนิดหน่วยลงทุน%20E'),
    ('SCBTMFPLUS-B', 'SCBAM-MMF-SCBTMFPLUS%20B-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน%20พลัส%20ชนิดหน่วยลงทุน%20B'),
    ('SCBTMFPLUS-A', 'SCBAM-MMF-SCBTMFPLUS%20A-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน%20พลัส%20ชนิดหน่วยลงทุน%20A'),
    ('SCBTMF', 'SCBAM-MMF-SCBTMF-กองทุนเปิดไทยพาณิชย์ตราสารรัฐตลาดเงิน'),
    ('SCBSFFPLUS-P', 'SCBAM-FIXSTB-SCBSFFPLUS%20P-กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น%20พลัส%20ชนิดหน่วยลงทุน%20P'),
    ('SCBSFFPLUS-I', 'SCBAM-FIXSTB-SCBSFFPLUS%20I-กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น%20พลัส%20ชนิดหน่วยลงทุน%20I'),
    ('SCBSFFPLUS-B', 'SCBAM-FIXSTB-SCBSFFPLUS%20B-กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น%20พลัส%20ชนิดหน่วยลงทุน%20B'),
    ('SCBSFFPLUS-A', 'SCBAM-FIXSTB-SCBSFFPLUS%20A-กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น%20พลัส%20ชนิดหน่วยลงทุน%20A'),
    ('SCBSFF', 'SCBAM-FIXSTB-SCBSFF-กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น'),
    ('SCBFIXEDA', 'SCBAM-FIXSTB-SCBFIXEDA-กองทุนเปิดไทยพาณิชย์ตราสารหนี้%20ชนิดสะสมมูลค่า'),
    ('SCBRF', 'SCBAM-FIXMTB-SCBRF-กองทุนเปิดไทยพาณิชย์เกษียณสุข%20%20ตราสารหนี้%20'),
    ('SCBFPP', 'SCBAM-FIXMTB-SCBFPP-กองทุนเปิดไทยพาณิชย์ตราสารหนี้%20พลัส%20%20ชนิดผู้ลงทุนกลุ่ม%20บุคคล%20'),
    ('SCBFP', 'SCBAM-FIXMTB-SCBFP-กองทุนเปิดไทยพาณิชย์ตราสารหนี้%20พลัส%20%20ชนิดสะสมมูลค่า%20'),
    ('BFIXED', 'BBLAM-FIXMTB-BFIXED-กองทุนเปิดบัวหลวงตราสารหนี้'),
    ('B-ENHANCED', 'BBLAM-FIXMTB-B%20ENHANCED-กองทุนเปิดบัวหลวงเพิ่มพูน'),
    ('B-TREASURY', 'BBLAM-MMF-B%20TREASURY-กองทุนเปิดบัวหลวงตราสารหนี้ภาครัฐ'),
    ('B-TNTV', 'BBLAM-FIXSTB-B%20TNTV-กองทุนเปิดบัวหลวงธนทวี'),
    ('TMBCB', 'TMBAM-FIXMTB-TMBCB-กองทุนเปิดทหารไทยธนวัฒน์'),
    ('TMBTM', 'TMBAM-MMF-TMBTM-กองทุนเปิดทหารไทยธนรัฐ'),
    ('TMBMF', 'TMBAM-MMF-TMBMF-กองทุนเปิดทหารไทยธนบดี'),
    ('ES-SPlus', 'TMBAM-FIXSTB-ES%20SPlus-กองทุนเปิดอีสท์สปริง%20ชอร์ทเทอม%20อินคัม%20พลัส'),
    ('K-PLAN1', 'KAsset-FIXMTB-K%20PLAN1-กองทุนเปิดเค%20แพลน%201'),
    ('K-FIXEDPLUS-A', 'KAsset-FIXMTB-K%20FIXEDPLUS%20A-กองทุนเปิดเคพลัส%20ตราสารหนี้%20พลัส%20ชนิดผู้ลงทุนทั่วไป'),
    ('K-CBOND-A', 'KAsset-FIXMTB-K%20CBOND%20A-กองทุนเปิดเค%20หุ้นกู้%20ชนิดผู้ลงทุนทั่วไป'),
    ('KACB', 'KAsset-FIXMTB-KACB-กองทุนเปิดเคเอ%20คอร์เปอเรท%20บอนด์'),
    ('K-TREASURY', 'KAsset-MMF-K%20TREASURY-กองทุนเปิดเค%20ตราสารรัฐระยะสั้น'),
    ('K-SFPLUS', 'KAsset-FIXSTB-K%20SFPLUS-กองทุนเปิดเค%20เอสเอฟ%20พลัส'),
    ('K-SF', 'KAsset-FIXSTB-K%20SF-กองทุนเปิดเค%20ตราสารหนี้ระยะสั้น'),
    ('K-MONEY', 'KAsset-MMF-K%20MONEY-กองทุนเปิดเค%20ตลาดเงิน'),
    ('K-CASH', 'KAsset-MMF-K%20CASH-กองทุนเปิดเค%20บริหารเงิน'),
    ('KASF', 'KAsset-FIXSTB-KASF-กองทุนเปิดเคเอ%20ชอร์ท%20เทอม%20ฟิกซ์%20อินคัม'),
    ('KTFIXPLUS-I', 'KTAM-FIXMTB-KTFIXPLUS%20I-กองทุนเปิดกรุงไทยตราสารหนี้%20พลัส%20ชนิดผู้ลงทุนสถาบัน'),
    ('KTFIXPLUS-D', 'KTAM-FIXMTB-KTFIXPLUS%20D-กองทุนเปิดกรุงไทยตราสารหนี้%20พลัส%20ชนิดจ่ายเงินปันผล'),
    ('KTFIXPLUS-A', 'KTAM-FIXMTB-KTFIXPLUS%20A-กองทุนเปิดกรุงไทยตราสารหนี้%20พลัส%20ชนิดสะสมมูลค่า'),
    ('KTFIX-1Y3Y', 'KTAM-FIXMTB-KTFIX%201Y3Y-กองทุนเปิดกรุงไทยตราสารหนี้ระยะ%201%20ถึง%203%20ปี'),
    ('KTPLUS', 'KTAM-FIXSTB-KTPLUS-กองทุนเปิดกรุงไทย%20ธนทรัพย์%20พลัส'),
    ('KTSS', 'KTAM-MMF-KTSS-กองทุนเปิดกรุงไทยตราสารตลาดเงิน'),
    ('KT-ST', 'KTAM-FIXSTB-KT%20ST-กองทุนเปิดกรุงไทยตราสารหนี้ระยะสั้น'),
    ('KTSTPLUS-A', 'KTAM-FIXSTB-KTSTPLUS%20A-กองทุนเปิดกรุงไทยตราสารหนี้ระยะสั้น%20พลัส%20ชนิดสะสมมูลค่า'),
    ('KTSV', 'KTAM-MMF-KTSV-กองทุนเปิดกรุงไทยตราสารตลาดเงินภาครัฐ')
]
len(bond_funds)

42

In [5]:
df.loc[df['retrun 3 month'] > 1].sort_values('retrun 3 month')

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
15,BFIXED,23344741208,0.00,0.50,0.80,False,1.01,-0.45,-0.73,0.48,1.03,10/08/2565
34,KTFIXPLUS-D,233784166,1.00,1.00,0.80,True,1.05,-0.62,-1.37,0.85,1.35,11/08/2565
35,KTFIXPLUS-A,2799179155,1.00,1.00,0.80,False,1.06,-0.61,-1.36,0.86,1.35,11/08/2565
23,K-PLAN1,14175999332,0.50,0.50,1.07,False,1.07,-0.19,-0.57,0.93,1.40,10/08/2565
33,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
12,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565
14,SCBFP,14816718949,1.07,1.07,1.66,False,1.56,-0.27,-0.59,0.95,1.35,10/08/2565
13,SCBFPP,38864193,1.07,1.07,1.12,False,1.66,-0.08,-0.22,0.84,1.44,10/08/2565
24,K-FIXEDPLUS-A,21103289689,0.00,0.00,1.07,False,1.88,-0.94,-2.15,0.20,0.12,10/08/2565


In [9]:
df.loc[df['retrun 1 year'] > 0].sort_values(['retrun 3 month', 'retrun 1 year'], ascending=[False, True])

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
33,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
11,SCBFIXEDA,5533196449,3.21,3.21,2.68,False,0.28,-0.01,0.16,0.99,0.00,10/08/2565
28,K-SFPLUS,77804127451,0.00,0.00,1.07,False,0.25,0.02,0.12,0.78,1.07,10/08/2565
39,KT-ST,8585282719,0.00,0.00,0.43,False,0.23,0.14,0.34,0.98,1.28,11/08/2565
32,KASF,934295832,3.00,3.00,0.11,False,0.19,0.05,0.38,1.07,1.34,10/08/2565
40,KTSTPLUS-A,32243801140,3.00,3.00,2.14,False,0.17,0.00,0.13,0.75,1.09,11/08/2565
6,SCBSFFPLUS-P,239839972,0.00,1.07,0.96,False,0.17,0.31,0.68,0.90,0.00,10/08/2565
37,KTPLUS,28457268639,1.00,1.00,1.07,False,0.14,0.04,0.19,0.53,0.80,11/08/2565
29,K-SF,150132535670,0.00,0.00,1.07,False,0.12,0.17,0.30,0.58,0.84,10/08/2565
2,SCBTMFPLUS-E,27529030,0.00,0.00,0.96,False,0.09,0.18,0.37,0.53,0.00,10/08/2565


In [10]:
df.nlargest(n=5, columns=['retrun 3 month', 'retrun 6 month', 'retrun 1 year', 'retrun 3 year', 'retrun 5 year'])

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
24,K-FIXEDPLUS-A,21103289689,0.00,0.00,1.07,False,1.88,-0.94,-2.15,0.20,0.12,10/08/2565
13,SCBFPP,38864193,1.07,1.07,1.12,False,1.66,-0.08,-0.22,0.84,1.44,10/08/2565
14,SCBFP,14816718949,1.07,1.07,1.66,False,1.56,-0.27,-0.59,0.95,1.35,10/08/2565
12,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565
33,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565


In [11]:
df.nsmallest(n=5, columns=['frontend fee', 'backend fee', 'management fee'])

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
20,TMBTM,16846476874,0.0,0.0,0.25,False,0.06,0.11,0.23,0.40,0.67,10/08/2565
39,KT-ST,8585282719,0.0,0.0,0.43,False,0.23,0.14,0.34,0.98,1.28,11/08/2565
22,ES-SPlus,1490185323,0.0,0.0,0.50,False,0.06,-0.24,0.00,0.00,0.00,10/08/2565
27,K-TREASURY,16428089812,0.0,0.0,0.54,False,0.04,0.07,0.14,0.40,0.63,10/08/2565
30,K-MONEY,41409409412,0.0,0.0,0.54,False,0.05,0.07,0.10,0.30,0.53,10/08/2565


In [12]:
print(f"mean: {df['management fee'].mean()}, min: {df['management fee'].min()}, max: {df['management fee'].max()}")

mean: 0.9307142857142856, min: 0.11, max: 2.68


In [13]:
tops_retrun_3m = df.nlargest(n=5, columns=['retrun 3 month']) # .loc[df['dividend yield'] == True]
tops_retrun_3m

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
24,K-FIXEDPLUS-A,21103289689,0.00,0.00,1.07,False,1.88,-0.94,-2.15,0.20,0.12,10/08/2565
13,SCBFPP,38864193,1.07,1.07,1.12,False,1.66,-0.08,-0.22,0.84,1.44,10/08/2565
14,SCBFP,14816718949,1.07,1.07,1.66,False,1.56,-0.27,-0.59,0.95,1.35,10/08/2565
12,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565
33,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565


In [14]:
tops_retrun_6m = df.nlargest(n=5, columns=['retrun 6 month'])
tops_retrun_6m

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
33,KTFIXPLUS-I,41686200,1.0,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
6,SCBSFFPLUS-P,239839972,0.0,1.07,0.96,False,0.17,0.31,0.68,0.90,0.00,10/08/2565
2,SCBTMFPLUS-E,27529030,0.0,0.00,0.96,False,0.09,0.18,0.37,0.53,0.00,10/08/2565
29,K-SF,150132535670,0.0,0.00,1.07,False,0.12,0.17,0.30,0.58,0.84,10/08/2565
21,TMBMF,5185344024,0.0,0.00,0.75,False,0.08,0.15,0.26,0.39,0.64,10/08/2565


In [15]:
tops_retrun_1y = df.nlargest(n=5, columns=['retrun 1 year'])
tops_retrun_1y

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
6,SCBSFFPLUS-P,239839972,0.0,1.07,0.96,False,0.17,0.31,0.68,0.90,0.00,10/08/2565
33,KTFIXPLUS-I,41686200,1.0,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
32,KASF,934295832,3.0,3.00,0.11,False,0.19,0.05,0.38,1.07,1.34,10/08/2565
2,SCBTMFPLUS-E,27529030,0.0,0.00,0.96,False,0.09,0.18,0.37,0.53,0.00,10/08/2565
39,KT-ST,8585282719,0.0,0.00,0.43,False,0.23,0.14,0.34,0.98,1.28,11/08/2565


In [16]:
tops_retrun_3y = df.nlargest(n=5, columns=['retrun 3 year'])
tops_retrun_3y

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
33,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
26,KACB,3997636778,3.00,3.00,0.11,False,0.92,-0.21,-0.03,1.38,1.80,10/08/2565
32,KASF,934295832,3.00,3.00,0.11,False,0.19,0.05,0.38,1.07,1.34,10/08/2565
12,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565
11,SCBFIXEDA,5533196449,3.21,3.21,2.68,False,0.28,-0.01,0.16,0.99,0.00,10/08/2565


In [19]:
# union all top 10 in each return

all_funds = pd.concat([tops_retrun_3m, tops_retrun_6m, tops_retrun_1y, tops_retrun_3y], ignore_index = True)
all_funds = all_funds.drop_duplicates()
all_funds

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date
0,K-FIXEDPLUS-A,21103289689,0.00,0.00,1.07,False,1.88,-0.94,-2.15,0.20,0.12,10/08/2565
1,SCBFPP,38864193,1.07,1.07,1.12,False,1.66,-0.08,-0.22,0.84,1.44,10/08/2565
2,SCBFP,14816718949,1.07,1.07,1.66,False,1.56,-0.27,-0.59,0.95,1.35,10/08/2565
3,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565
4,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565
6,SCBSFFPLUS-P,239839972,0.00,1.07,0.96,False,0.17,0.31,0.68,0.90,0.00,10/08/2565
7,SCBTMFPLUS-E,27529030,0.00,0.00,0.96,False,0.09,0.18,0.37,0.53,0.00,10/08/2565
8,K-SF,150132535670,0.00,0.00,1.07,False,0.12,0.17,0.30,0.58,0.84,10/08/2565
9,TMBMF,5185344024,0.00,0.00,0.75,False,0.08,0.15,0.26,0.39,0.64,10/08/2565
12,KASF,934295832,3.00,3.00,0.11,False,0.19,0.05,0.38,1.07,1.34,10/08/2565


In [22]:
# for each fund in each return to get point: 5y = 5, 3y = 4, 1y = 3, 6m = 2, 3m =1
all_funds['point'] = 0

for index, row in all_funds.iterrows():
    
    for i , r in tops_retrun_3m.iterrows():
        if row['name'] == r['name']:
            all_funds.at[index, 'point'] = all_funds.at[index, 'point'] + 1
            
    for i , r in tops_retrun_6m.iterrows():
        if row['name'] == r['name']:
            all_funds.at[index, 'point'] = all_funds.at[index, 'point'] + 2
            
    for i , r in tops_retrun_1y.iterrows():
        if row['name'] == r['name']:
            all_funds.at[index, 'point'] = all_funds.at[index, 'point'] + 3
            
    for i , r in tops_retrun_3y.iterrows():
        if row['name'] == r['name']:
            all_funds.at[index, 'point'] = all_funds.at[index, 'point'] + 4
            
#     print(f"{row['name']}, point: {all_funds.at[index, 'point']}")
            
all_funds.sort_values('point',  ascending=False)

,name,size,frontend fee,backend fee,management fee,dividend yield,retrun 3 month,retrun 6 month,retrun 1 year,retrun 3 year,retrun 5 year,updated date,point
4,KTFIXPLUS-I,41686200,1.00,1.00,0.80,True,1.09,1.29,0.61,1.62,1.84,11/08/2565,10
12,KASF,934295832,3.00,3.00,0.11,False,0.19,0.05,0.38,1.07,1.34,10/08/2565,7
3,SCBRF,804904743,5.00,0.00,1.04,False,1.12,-0.30,-0.44,1.05,1.48,10/08/2565,5
6,SCBSFFPLUS-P,239839972,0.00,1.07,0.96,False,0.17,0.31,0.68,0.90,0.00,10/08/2565,5
7,SCBTMFPLUS-E,27529030,0.00,0.00,0.96,False,0.09,0.18,0.37,0.53,0.00,10/08/2565,5
16,KACB,3997636778,3.00,3.00,0.11,False,0.92,-0.21,-0.03,1.38,1.80,10/08/2565,4
19,SCBFIXEDA,5533196449,3.21,3.21,2.68,False,0.28,-0.01,0.16,0.99,0.00,10/08/2565,4
14,KT-ST,8585282719,0.00,0.00,0.43,False,0.23,0.14,0.34,0.98,1.28,11/08/2565,3
8,K-SF,150132535670,0.00,0.00,1.07,False,0.12,0.17,0.30,0.58,0.84,10/08/2565,2
9,TMBMF,5185344024,0.00,0.00,0.75,False,0.08,0.15,0.26,0.39,0.64,10/08/2565,2


In [24]:
def getResponseSoup(url):
    res = requests.get(url)
    res.encoding = "utf-8"
    if res.status_code == 200:
        pass
#         print("Successful")
    elif res.status_code == 404:
        print("Error 404 page not found")
    else:
        print("Not both 200 and 404")
        
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

In [114]:
targeted_names = ['KTFIXPLUS-I', 'KASF', 'SCBRF']
df = pd.DataFrame()

for name, symbol in bond_funds:
    if name in targeted_names:
#         print(symbol)
        port_url = f"https://www.wealthmagik.com/FundInfo/FundPort-{symbol}"
        port_soup = getResponseSoup(port_url)

        fund_script = port_soup.find_all("script")
        fund_script[-8]
        top5_raw = fund_script[-8].string.split('=')[1][1:-2].split('},')

        top5_raw = list(map(lambda x: x+'}' if x[-1] != '}' else x, top5_raw))
        top5_raw = list(map(lambda x: x.replace('title', '"title"').replace('value', '"value"').replace('\'', '"'), top5_raw))

        top5_holding = list(map(lambda x: json.loads(x), top5_raw))
        symbol_df = pd.DataFrame(top5_holding)
        symbol_df['value'] = symbol_df['value'].astype('float64')
        symbol_df['name'] = name
        
        df = pd.concat([df, symbol_df], ignore_index = True)
        
df

,title,value,name
0,ตราสารหนี้ธนาคาร EMIRATES NBD BANK PJSC,7.28,SCBRF
1,หุ้นกู้ของบริษัท ทรูคอร์ปอเรชั่น จำกัด (มหาชน)...,6.75,SCBRF
2,หุ้นกู้ส่งเสริมความยั่งยืนของ บริษัท อินโดรามา...,6.53,SCBRF
3,หุ้นกู้ของบริษัท ศุภาลัย จำกัด (มหาชน) ครั้งที...,5.41,SCBRF
4,หุ้นกู้ของบริษัท ไมเนอร์อินเตอร์เนชั่นแนล จำกั...,3.99,SCBRF
5,Other,70.04,SCBRF
6,พันธบัตรธนาคารแห่งประเทศไทย (BOT BOND) (AAA),37.78,KASF
7,ธนาคารอาคารสงเคราะห์ (GHB) (AAA(tha)),8.11,KASF
8,บมจ.เงินติดล้อ (TIDLOR) (A),7.59,KASF
9,บมจ.ไทยเบฟเวอเรจ (TBEV) (AA(tha)),5.78,KASF


### Test code below

In [40]:
symbol = 'KTAM-FIXMTB-KTFIXPLUS%20I-กองทุนเปิดกรุงไทยตราสารหนี้%20พลัส%20ชนิดผู้ลงทุนสถาบัน'
port_url = f"https://www.wealthmagik.com/FundInfo/FundPort-{symbol}"
port_soup = getResponseSoup(port_url)

fund_script = port_soup.find_all("script")
fund_script[-8]

<script>chartTopHoldingData=[{title:'พันธบัตรรัฐบาลเพื่อการบริหารหนี้ ในปีงบประมาณ พ.ศ. 2564 ครั้งที่ 1',value:'5.27'},{title:'พันธบัตรรัฐบาล ในปีงบประมาณ พ.ศ. 2553 ครั้งที่ 2',value:'4.21'},{title:'เงินฝากประจำ DOHA BANK QPSC/240222',value:'3.82'},{title:'หุ้นกู้ของบริษัทบริหารสินทรัพย์ กรุงเทพพาณิชย์ จำกัด (มหาชน) ครั้งที่ 1/2563 ชุดที่ 2',value:'3.78'},{title:'พันธบัตรรัฐบาลเพื่อการปรับโครงสร้างหนี้ปีงบประมาณ พ.ศ. 2562 ครั้งที่ 4',value:'3.73'},{title:'Other',value:'79.19'}];</script>

In [65]:
top5_raw = fund_script[-8].string.split('=')[1][1:-2].split('},')

top5_raw = list(map(lambda x: x+'}' if x[-1] != '}' else x, top5_raw))
top5_raw = list(map(lambda x: x.replace('title', '"title"').replace('value', '"value"').replace('\'', '"'), top5_raw))
top5_raw

['{"title":"พันธบัตรรัฐบาลเพื่อการบริหารหนี้ ในปีงบประมาณ พ.ศ. 2564 ครั้งที่ 1","value":"5.27"}',
 '{"title":"พันธบัตรรัฐบาล ในปีงบประมาณ พ.ศ. 2553 ครั้งที่ 2","value":"4.21"}',
 '{"title":"เงินฝากประจำ DOHA BANK QPSC/240222","value":"3.82"}',
 '{"title":"หุ้นกู้ของบริษัทบริหารสินทรัพย์ กรุงเทพพาณิชย์ จำกัด (มหาชน) ครั้งที่ 1/2563 ชุดที่ 2","value":"3.78"}',
 '{"title":"พันธบัตรรัฐบาลเพื่อการปรับโครงสร้างหนี้ปีงบประมาณ พ.ศ. 2562 ครั้งที่ 4","value":"3.73"}',
 '{"title":"Other","value":"79.19"}']

In [90]:
top5_holding = list(map(lambda x: json.loads(x), top5_raw))
top5_holding

df = pd.DataFrame(top5_holding)
df['value'] = df['value'].astype('float64') 
df

,title,value
0,พันธบัตรรัฐบาลเพื่อการบริหารหนี้ ในปีงบประมาณ ...,5.27
1,พันธบัตรรัฐบาล ในปีงบประมาณ พ.ศ. 2553 ครั้งที่ 2,4.21
2,เงินฝากประจำ DOHA BANK QPSC/240222,3.82
3,หุ้นกู้ของบริษัทบริหารสินทรัพย์ กรุงเทพพาณิชย์...,3.78
4,พันธบัตรรัฐบาลเพื่อการปรับโครงสร้างหนี้ปีงบประ...,3.73
5,Other,79.19
